In [4]:
import os
# 【硬性要求】加载环境变量 (确保可以调用 OpenAI 和 Tavily API)
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 【新增引入】从实验包中引入 Python REPL 工具
from langchain_experimental.tools import PythonREPLTool

# ==========================================
# 1. 组装多工具箱 (Assembling the Toolbelt)
# ==========================================
# 搜索引擎工具 (Agent 的"眼睛")
search_tool = TavilySearchResults()

# Python 解释器工具 (Agent 的"计算器")
python_tool = PythonREPLTool()

# 将两个工具同时放入工具箱列表中
tools = [search_tool, python_tool] 

# ==========================================
# 2. 构建核心 Agent (Building the Agent)
# ==========================================
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Think step by step before responding."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_tool_calling_agent(llm, tools, prompt)

# 开启 verbose=True，等下运行的时候你会惊叹于它在后台的思考过程！
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# ==========================================
# 3. 注入长期记忆 (Injecting Memory)
# ==========================================
# (保持上一节课的现代工厂函数写法，防止多并发串智)
store = {}
def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

agent_with_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# ==========================================
# 4. 执行终极测试 (Execution)
# ==========================================
if __name__ == "__main__":
    config = {"configurable": {"session_id": "lab_session_math_test"}}
    
    # 模拟老师讲稿中的高难度复合查询：设定当前日期，要求查询开赛日并计算差值
    # Agent 会先用 Tavily 查开赛日，再用 Python 算日期差
    complex_query = "Today's date is May 1st, 2024. How many days before the ICC Men's T20 World Cup first match starts?"
    
    print(f"=== 🙋 发起复合任务: {complex_query} ===")
    print("请观察终端中绿色的思考过程 (Thought Process)...\n")
    
    response = agent_with_history.invoke({"input": complex_query}, config=config)
    
    print(f"\n=== 🤖 最终完美回答 ===")
    print(response['output']) # 它会告诉你精确的天数，彻底消除数学幻觉

=== 🙋 发起复合任务: Today's date is May 1st, 2024. How many days before the ICC Men's T20 World Cup first match starts? ===
请观察终端中绿色的思考过程 (Thought Process)...



> Entering new AgentExecutor chain...

Invoking: `Python_REPL` with `{'query': "from datetime import datetime\nfrom dateutil import relativedelta\n\ntoday = datetime.strptime('2024-05-01', '%Y-%m-%d')\nworld_cup_start = datetime.strptime('2024-10-18', '%Y-%m-%d')\n\ndifference = relativedelta.relativedelta(world_cup_start, today)\ndays_before_start = difference.days\n\ndays_before_start"}`


The ICC Men's T20 World Cup first match starts in 170 days from today's date, which is on October 18, 2024.

> Finished chain.

=== 🤖 最终完美回答 ===
The ICC Men's T20 World Cup first match starts in 170 days from today's date, which is on October 18, 2024.
